In [ ]:
using Convex
using GLPK
using Printf

# Introduction to convex optimization

\begin{equation}
\label{Eq:Program}
  \begin{array}{rll}
    \text{minimize} & f (\vec{x}) & \\
    \text{over} & \vec{x} \in R^n & \\
    & g_j (\vec{x}) \leq 0 & j \in \mathcal{J}\\
    & h_k (\vec{x}) = 0 & k \in \mathcal{K}
  \end{array}
\end{equation}

where all $f$, $g_j$ and $h_k$ are convex; for $f(\vec{x})$:

\begin{equation}
  \label{Eq:Convex}
  f (\alpha \vec{x}_1 + (1 - \alpha) \vec{x}_2) \leqslant  \alpha f (\vec{x}_1) + (1 - \alpha) f (\vec{x}_2), \qquad \alpha \in [0, 1]
\end{equation}

On left, an example of a convex function is given below, where we plot the line segment $\color{blue}{\alpha f (\vec{x}_1) + (1 - \alpha) f (\vec{x}_2)}$ in dark blue. On the right, an example of a nonconvex function.

![ConvexFun.svg](ConvexFun.svg)

In the same spirit, convex sets include the line segment between any two of their points:
![Convex_polygon_illustration1.svg](Convex_polygon_illustration1.svg)
while convex sets do not:
![Convex_polygon_illustration2.svg](Convex_polygon_illustration2.svg)

Convex programs \eqref{Eq:Program} have one crucial property: any local minimum is also a global minimum. In particular, this guarantees that (non pathological) optimization algorithms always find the global minimum up to numerical precision.

## Linear optimization

The simplest kind of convex programs is a *linear program*, where the objective and all constraints are represented by linear functions such as (technically, these are *affine functions*):

\begin{equation}
f(\vec{x}) = \vec{c}^\top \vec{x} + d
\end{equation}


\begin{equation}
\label{Eq:LPExample}
  \begin{array}{rl}
    p^\star = \text{minimize} & x_1 + 2 x_2 - x_3 \\
    \text{over} & \vec{x} \in R^3 \\
    & x_1, x_2, x_3 \ge 0 \\
    & x_1 + x_2 + x_3 = 1
  \end{array}
\end{equation}

We now solve that problem using the [Convex.jl](https://github.com/JuliaOpt/Convex.jl) optimization toolbox.

In [ ]:
x = Variable(3)
constraints = [x >= 0, sum(x) == 1]
objective = x[1] + 2*x[2] - x[3]
problem = Convex.minimize(objective, constraints)
solve!(problem, GLPK.Optimizer(msg_lev=GLPK.MSG_ALL ))
@printf("\n")
@printf("Optimal objective p = %f\n", problem.optval)
@printf("Optimal point     x = %s\n", x.value)

### Primal canonical form of a linear program

The standard form of a linear program is as follows, with $\vec{c} \in \mathbb{R}^n$, $A \in \mathbb{R}^{m \times n}$ and $\vec{b} \in \mathbb{R}^m$.

\begin{equation}
\label{Eq:PrimalForm}
  \begin{array}{rl}
    \text{minimize} & \vec{c}^\top \vec{x} \\
    \text{over} & \vec{x} \in R^n \\
    & A \vec{x} = \vec{b} \\
    & \vec{x} \ge 0
  \end{array}
\end{equation}

The problem is fully specified by the matrix $A$ and the vectors $\vec{b}$ and $\vec{c}$.

#### Lecture exercice

Complete the program data below to solve \eqref{Eq:LPExample}.

In [ ]:
using Convex
using GLPK
# remember [1, 1, 1] is a 1D vector, [1; 1; 1] is a column matrix, [1 1 1] is a row matrix
x = Variable(3)
c = []
A = []
b = []
constraints = [A*x == b, x >= 0]
objective = dot(c, x)
problem = Convex.minimize(objective, constraints)
solve!(problem, GLPK.Optimizer(msg_lev=GLPK.MSG_ALL))
@printf("\n")
@printf("Optimal objective p = %f\n", problem.optval)
@printf("Optimal point     x = %s\n", x.value)

## Duality

We remind the primal canonical form \eqref{Eq:PrimalForm}:
\begin{equation*}
  p^\star = \underset{\vec{x} \in R^n }{\text{minimize}} \quad \vec{c}^\top \vec{x}, \qquad A \vec{x} = \vec{b}, \qquad \vec{x} \ge 0,
\end{equation*}

and introduce the notation

- a variable with a $\star$ superscript is an optimal solution (as in $p^\star$ is the minimum of the LP),
- a variable with a $*$ superscript is a feasible solution (as in $\vec{x}^*$ satisfies the constraints),

with the conventions

- $p^\star = + \infty$ if the problem is infeasible,
- $p^\star$ is finite if the problem has an optimal (thus feasible) solution,
- $p^\star = -\infty$ if the problem is unbounded, as there are feasible solutions with $\vec{c}^\top \vec{x} \rightarrow -\infty$.

We introduce the Lagrange multipliers $\vec{y} \in \mathbb{R}^m$ corresponding to the constraint $A \vec{x} = \vec{b}$, and the Lagrange multipliers $\vec{s} \in \mathbb{R}^n$ corresponding to the constraint $\vec{x} \ge 0$, with $\vec{s} \ge 0$ as they correspond to an inequality constraint. The Lagrangian function is:

\begin{equation}
\label{Eq:Lagrangian}
L(\vec{x},\vec{y},\vec{s}) = \vec{c}^\top x + \vec{y}^\top (\vec{b} - A \vec{x}) - \vec{s}^\top \vec{x}.
\end{equation}

For any feasible $\vec{x}^*$ and any $(\vec{y}^*, \vec{s}^*)$ with $\vec{s} \ge 0$, we have

\begin{equation}
\label{Eq:LowerBound}
L(\vec{x}^*,\vec{y}^*,\vec{s}^*) = \vec{c}^\top x^* + (\vec{y}^*)^\top \underbrace{(\vec{b} - A \vec{x}^*)}_{=0} - (\vec{s}^*)^\top \vec{x}^* \le \vec{c}^\top \vec{x}^*
\end{equation}

Let us now minimize $L(\vec{x}, \vec{y}, \vec{s})$ over $\vec{x}$:

\begin{equation}
\label{Eq:DualCases}
g(\vec{y}, \vec{s}) = \min_{\vec{x}} \vec{x}^\top (\vec{c} - A^\top \vec{y} - \vec{s}) + \vec{b}^\top \vec{y} = \begin{cases} \vec{b}^\top \vec{y}, & \vec{c} - A^\top \vec{y} - \vec{s} = 0, \\ -\infty, & \text{otherwise}. \end{cases}
\end{equation}

Note that for every $(\vec{y}, \vec{s})$ (with $\vec{s} \ge 0$ by definition), the value $g(\vec{y}, \vec{s})$ is a lower bound for $p^\star$ by \eqref{Eq:LowerBound}.

In \eqref{Eq:DualCases}, only the first case ($\vec{c} - A^\top \vec{y} - \vec{s} = 0$) leads to a nontrivial upper bound. We now maximize over such $(\vec{y}, \vec{s})$ to get the best lower bound.

### Dual canonical form of a linear program

We obtain:

\begin{equation}
\label{Eq:DualForm}
  \begin{array}{rl}
    d^\star = \text{maximize} & \vec{b}^\top \vec{y} \\
    \text{over} & \vec{y} \in \mathbb{R}^m, \vec{s} \in \mathbb{S}^n \\
    & \vec{c} - A^\top \vec{y} = \vec{s} \\
    & \vec{s} \ge 0
  \end{array}
\end{equation}

which is the dual problem. Through the Lagrangian mechanism, we have $d^\star \le p^\star$ which is *weak duality*.

We have the cases, for the dual problem:
- $d^\star = +\infty$, dual unbounded, thus $+\infty \le p^\star$ and the primal problem is infeasible,
- $d^\star$ is finite, the dual is feasible, the primal problem can either be feasible or infeasible (but not unbounded),
- $d^\star = -\infty$, the dual problem is infeasible.

For linear programs only, when $d^\star$ is finite, then $p^\star = d^\star$; when $p^\star$ is finite, also $d^\star = p^\star$ (this is called *strong duality*).

#### Lecture exercice

Fill the problem data below to solve the dual of \eqref{Eq:LPExample}.

In [ ]:
using Convex
using GLPK
y = Variable(1)
s = Variable(3)
c = []
A = []
b = []
constraints = [s >= 0, s == c - transpose(A)*y]
objective = dot(b, y)
problem = Convex.maximize(objective, constraints)
solve!(problem, GLPK.Optimizer(msg_lev=GLPK.MSG_ALL))
@printf("\n")
@printf("Optimal objective d = %f\n", problem.optval)
@printf("Optimal point     y = %s\n", y.value)
@printf("Optimal point     s = %s\n", s.value)

### Recovering dual variables using Convex.jl

Most convex solvers solve the primal and the dual problem at the same time, as doing so lead to better robustness (the primal-dual gap $p^\star - d^\star$ is then a measure of convergence).

Remember the primal problem:
\begin{equation*}
  p^\star = \underset{\vec{x} \in R^n }{\text{minimize}} \quad \vec{c}^\top \vec{x}, \qquad A \vec{x} = \vec{b}, \qquad \vec{x} \ge 0,
\end{equation*}
and realize that any feasible $\vec{x}^*$ provides an objective $p^*$ that is an upper bound on the true minimum by definition: $p^\star \le p^*$.

The same happens for the dual problem: any feasible pair $(\vec{y}^*, \vec{s}^*)$ provides an objective $d^*$ that is a lower bound on the true dual maximum.

Thus if we have a primal feasible solution $\vec{x}^*$ and a dual feasible solution $(\vec{y}^*, \vec{s}^*)$, we sandwich the true solution $d^\star = p^\star$:

\begin{equation}
\label{Eq:Sandwich}
d^* \le d^\star = p^\star \le p^*.
\end{equation}

Linear programming solvers try to return a feasible solution pair for both the primal and dual, with primal objective $\tilde{p}^*$ and dual objective $\tilde{d}^*$. It would be tempting to imagine that the true solution lies in the internal $[\tilde{d}^*, \tilde{p}^*]$. However the primal and dual solutions are usually *slightly* infeasible due to numerical errors, and the relation \eqref{Eq:Sandwich} only holds approximately. Worse, it's difficult to assess the impact of such numerical errors just by looking at the problem data.

Below, we provide the primal problem to Convex.jl, and recover the dual variables from the solution. Compare and contrast with the dual formulation above.

In [ ]:
using Convex
using GLPK
x = Variable(3)
c = []
A = []
b = []
constraints = [A*x == b, x >= 0]
objective = dot(c, x)
problem = Convex.minimize(objective, constraints)
solve!(problem, GLPK.Optimizer(msg_lev=GLPK.MSG_ALL ))
@printf("\n")
@printf("Optimal objective d = %f\n", problem.optval)
@printf("Optimal point     x = %s\n", x.value)
@printf("Dual variable 1   %s\n", problem.constraints[1].dual)
@printf("Dual variable 2   %s\n", problem.constraints[2].dual)